# Why is Multi-Task Learning Research Interesting?

In machine learning we often have this implicit notion of a **task** that we are training our models to perform &mdash; a model trained on MNIST data is trained to perform the task of _handwritten digit recognition_, a model trained on CoNLL NER is learning to perform _named entity recognition_.

But what _really_ is the task of handwritten digit recognition? How would we define such a task?

## Table of Contents

1. What is a Task?
2. Multi-Task Learning in deep learning.
3. Where does negative transfer come from?
   1. The Loss Function
   2. Generalization to IID
   3. Underspecification
4. How do we mitigate negative transfer?
5. Which tasks should be learnt together?

## 1. What is a Task?

In machine learning, we often have this implicit notion of some task that our model is learning.



Let's take a first stab at defining a task in machine learning.
This almost always begins with the assumption that our task consists of an input and output space, say $\mathcal{X}$ and $\mathcal{Y}$ respectively.
Rolling with MNIST as an example, here we would say that $\mathcal{X}_{mnist}$ is all $28\times28$ grey-scale images and $\mathcal{Y}_{mnist}$ is a class label from $0-9$. Below are some examples of the MNIST dataset.

![](blog_figs/what_is_a_task/mnist.png)

What's clear immediately is that this is not sufficient to define a task - for instance, take the FashionMNIST task. This task consists of classifying input images into 10 different articles of clothing. It _also_ consists of $28\times28$ grey-scale images and it's output space is also a class label from $0-9$. Below are some examples of FashionMNIST images.

![](blog_figs/what_is_a_task/fashion-mnist.png)

So FashionMNIST would also be defined by $\mathcal{X}_{mnist}$ and $\mathcal{Y}_{mnist}$. However, we (you & I) have a prior belief that FashionMNIST and MNIST are _not_ the same task (at least, I'm assuming you do). MNIST is "hand-digit recognition" and FashionMNIST is "clothing article recognition". So what's missing?

We could say that a task is defined by it's _distribution_ over $(\mathcal{X}, \mathcal{Y})$.
For example, MNIST defines a distribution $\mathcal{D}_{mnist} (\mathcal{X}_{mnist}, \mathcal{Y}_{mnist})$ that has high support for examples $(x, y)$ where $x$ is a $28\times28$ image of a hand-written digit and $y$ is the corresponding class label for that digit, and low (or no) support for examples where $x$ is not an image of a hand-written digit or $y$ is not the correct corresponding class label. Similarly, FashionMNIST now gets it's own distribution $\mathcal{D}_{fashion}(\mathcal{X}_{mnist}, \mathcal{Y}_{mnist})$ which has no support for images that look like handwritten digits, and high support for images containing articles of clothing.
So even though the two tasks have the same $\mathcal{X}$ and $\mathcal{Y}$, they are differentiated by both the
distribution over $\mathcal{X}$, as well as the distribution over $\mathcal{Y} | \mathcal{X}$.

So, now we have a way to define our tasks. We need:
- An input space $\mathcal{X}$
- An output space $\mathcal{Y}$
- And a joint distribution over those spaces, $\mathcal{D}(\mathcal{X}, \mathcal{Y})$

## The Generating Function Perspective

Let's instead take a different perspective of what a task is.

Let's instead assume that their is some generative function, $f$ which generates the labels that we are interested in.
We could instead say that a 

## The Model Function Perspective

## Why Machine Learning Doesn't Care

Talk about hidden artifacts, the _function_ that is learned.



#### An overly formal definition of a "task"

What do you consider a task? Perhaps a set of instructions? A command? Just... any _thing_ that needs to be done? In my mind the word "task" has something of a negative connotation, like it's any thing I don't want to do but I need to do.

In machine learning we have a bit more of a restrictive definition because we have to make certain architectural and optimization decisions in order to... well start somewhere!
Let me try to formally define a "task" in machine learning.
We might say that a task is comprised of:

- an input space, $X$
- an output space, $Y$
- a "labeling function" $f_t: X \rightarrow Y$, which outputs the ground truth label $y$ for task $t$ given any input $x$.

This function $f_t$ is the "human" part of a task. It's the piece that represents a certain part of our world that we wish for the model to learn. It's the map between the input space

I'm being ambiguous on purpose - machine learning has many datasets that are annotated by crowd-workers, and many datasets that are annotated by experts. Who decides what level of expertise is needed to perform a task? Well... certainly not me. In the MNIST dataset $f_t$ represents a mapping of images that look somewhat like a digit to a specific label representing that digit. In a semantic parsing dataset, $f_t$ represents a group of experts opinion of what "semantics" means, and how it should be represented in language, by way of several expertly annotated examples. 

In machine learning, to train our (supervised) models we require _(supervised) datasets_.
Datasets contain the following:
- an input space, $X$, and a distribution over the input space $D(X)$ from which examples are sampled.
- an output space, $Y$ and a gold-truth label for each sample.
- a "labeling function" $f_t: X \rightarrow Y$ which represents the way in which the gold-truch labels were obtained.



Here we are being somewhat loose with our definition of $f$ - what does $f$ _truly_ represent? If we are training ML model that we wish to model some aspect of our world, then perhaps $f$ represents some "human behavior" associated with $t$, i.e. perhaps the "average human response" when prompted with a task definition and input, or perhaps an "expert's response". $f$ need not necessarily be deterministic. If we wish to allow for e.g. alleatoric uncertainty, we could let $f$ be a stochastic labeling function, representing the distribution $P(y|x)$.

For example, let's say we are interested in the "task" of labeling hand-written digits (i.e. MNIST).
Then we have that

- $X$ is the space of all 28x28 greyscale images.
- $Y$ is the space of integers $\{0, \ldots, 9\}$.
- $f_t$ is some function that represents an "average human" belief as to what digit is contained in the image.

Hmmm... that's an interesting definition of $f_t$. We are assuming that $f_t$ encodes some level of world knowledge, e.g. what a 0 typically looks like and what a handwritten 0 _can_ look like, and how that compares to other digits - i.e. what we expect an average human to know. We expect $f_t$ to leverage this knowledge to output _reasonable answers_ to a given 28x28 greyscale image. For example, say we get an image that looks like a cross between a 6 and a 5, such that 50% of people believe it is a 5 and 50% of people believe it is a 6 - then $f_t$ should output a 5 half of the time we query it on that input, and a 6 the other half. What happens if the image is not of a digit? Well, $f$ could perhaps have an "I don't know" option - or it could _say_ I don't know by just stochastically representing the uniform distribution over all possible labels, e.g. we have an equal chance to get any digit from $f$ given a non-sense input.

#### Changing the input space

What happens if we change the input space? For example, let's consider the following task:
- $X$ is the space of all 32x32 RGB images.
- $Y$ is the space of integers $\{0, \ldots, 9\}$.
- $f_t$ is some function that represents an "average human" belief as to what digit is contained in the image.

Is this the same task as we defined above? Well... I believe that _most people_ would assume it is. But that's perhaps only because we were loose with our definition of the task. What if we defined the task to be "some function that represents an "average human" belief as to what digit is contained in the 28x28 greyscale image".

In machine learning, a lot of "tasks" are defined by their datasets. A dataset not only defines a certain input space and output space for a given task, but also a distribution over those spaces. However, take a human - 

Now, let's consider the FashionMNIST task. FashionMNIST could be defined with the following variables:
- $X$ is the space of all 28x28 greyscale images.
- $Y$ is the space of integers $\{0, \ldots, 9\}$.
- $f_t$ is some function that represents an "average human" belief as to what clothing item (out of 9 static options) is contained in the image.


#### Changing the output space


#### Changing the ground truth function

Does the output space define the task? Ehhhh.... not exactly. Let's say that instead of MNIST we consider Fashion-MNIST. What changes between these datasets? The input space and output space are exactly the same - 28x28 grey-scale images and a label set 0-9. 2 things of interest change - the _distribution_ over the input space $X$ and the labeling function $f$, which we now assume to have some understanding of what different articles of clothes look like, rather than handwritten digits. So the output space cannot define the task. We can go a couple steps further even - what if we instead define a task that has the same distribution as MNIST, but the output is instead to output 9 - x, where x is the digit shown in the image. Is this the same task as MNIST? In some senses, no - our "golden labeling function" $f$, being restricted to some notion of "human behavior", would do something like identify the digit in the image (if it is a valid digit) and then subtract it from 9. What does a machine learning model do, though? To a lot of machine learning models, the labels are arbitrary - representing the task as 9-x instead of x is just a reflection of indices. The tasks are the _exact same_ task. They can be boiled down still to the identity of the handwritten digit. Why? Well... I suppose it has something to do with how there's the hard part "identifying the digit" and then the easy part, which is a simple deterministic map from one "index" to another. The deterministic map doesn't matter to an ML model - it's still performing the same task. And $f$ is performing that task under the hood as well - it just adds a simple step after it...

#### Combining tasks

Why does any of this rambling matter?

Recently I have been struggling with the following definition of a task:
- $X$ is the space of all possible sequences over a finite (but large) vocabulary.
- $Y$ is the space of all possible sequences over a finite (but large) vocabulary.
- $f_t$ is some function that represents an "average human" completion of the sequence X.

The task defined above is called language modeling - language models are trained to be able to generate sequences from a vocabulary that more or less (less, so far) represent the distribution of language that is output by humans. It can be an individual human, or a conversation between two humans, or a discussions between multiple humans. Often, we might even try to train a model on all of these distributions simultaneously and hope it implicitly learns to distinguish between the different settings.

Why am I struggling with it? The distribution of human language is very broad and encompases nearly any setting under which you can imagine language being used to communicate something.

For example in a regular conversation:

"Hey - can you translate 'Today I am going to the library because I forgot my ID there, like a dummy' into spanish?"

"Sure! Hoy, voy a ir a la biblioteca porque..."

Or maybe in a standardized test setting:

"Please summarize the following paragraph: Omar was sitting on the couch playing video games when the phone next to him begun to vibrate. ... "

"Okay, so the TL;DR is that Omar really didn't want to see Susan so he made up an excuse..."

Do you see my conundrum here? We, as humans, might neatly categorize these two sentences as separate tasks - translation (from english into spanish) and summarization. As humans, we might recognize that these tasks require slightly different abilities; For example you do not need to know any spanish vocabulary to summarize an english paragraph but you do to translate into spanish. You might need a deeper understanding of a given passage to summarize it than to translate it, or the needs of the person asking for the summarization.

However, are they really separate tasks? To a langauge model all that matters (in theory) is that they are both "valid sequences" from the perspective of general english languages. They could both be spoken by reasonable humans, and so they should have support from the distribution defined by a language model. As such, a language model should be able to output each response sentence, given the query sentence. So what makes them separate "tasks"?

We could attempt to separate them into different distributions of language - e.g. a mode of human language that represents someone performing a "translation task" and another mode that represents someone performing a "summarization task".

Why does any of this matter? Ultimately, I'm not trying to say anything grandiose here - there's no punchline.
If anything, it's like a reverse punch-line. I just tried to define a "task" several times to arrive at something satisfying and failed.
I just think it's interesting to think about. Ultimately... there are always two "tasks" at play. The task that we, as humans, imperfectly defined and the task that the machine learning model is _actually_ learning. And history has taught us that these are actually rarely aligned.

As we shift to more general types of models, trained on vast quantities of... observations about our world, I think it's important to keep this in mind. We might be inclined to believe that GPT-3 understands what translation is, because it is capable of 


Note that language modeling is not necessarily the only setting where this arises. For example, let's say we convert our distribution over human language into images of the text - that is, given an image containing some text.

What's the rub here? In a nutshell, it is because we are using language to describe the task as a part of the input. When we combine a very flexible input space with a very flexible output space and a very general labeling function, we can often squeeze any definition of a "task" into this setting without changing any part of the "task".

Well... one extremely important ingredient is language - language is flexible, it is general, it is how I am communicating my ideas to you now. It is not a silver bullet - it is probably impossible to describe a 4K resolution image of a weather map for example, such that 

## Changing Spaces

Problem with fixing $\mathcal{X}$ and $\mathcal{Y}$.

## Conclusion